<a href="https://colab.research.google.com/github/Sarvesh1814/HateXplain/blob/Sarvesh/CNN_GRU_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GRU, Dense, Dropout
import numpy as np

from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from gensim.models import Word2Vec
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
import tensorflow as tf


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import spacy
import nltk
from tensorflow.keras.utils import pad_sequences
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, Conv1D, MaxPooling1D, GRU, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

In [ ]:
# Download the pre-trained GloVe word embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2023-04-11 10:10:11--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-11 10:10:11--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-11 10:10:12--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Reproducibility/Sample Model/HateXplain2.csv")

In [ ]:
data = df[['post_tokens',	'Target_cat',	'Label_cat',	'final_rationales']]
data['post_tokens'] = data['post_tokens'].apply(lambda x: eval(x))
for i in range(len(data)):
  
  sentence =""
  for j in (data['post_tokens'].iloc[i]):
    
    sentence += j +" "
    
  data['post_tokens'].iloc[i] = sentence

<ipython-input-19-5f0ee8932d8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['post_tokens'] = data['post_tokens'].apply(lambda x: eval(x))
<ipython-input-19-5f0ee8932d8a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['post_tokens'].iloc[i] = sentence


In [ ]:
label_map = {'hatespeech': 0, 'normal': 1, 'offensive': 2}
labels = data["Label_cat"].apply(lambda x: label_map[x])

In [ ]:
max_words = 10000
maxlen = 100

# Tokenize the text data and convert it to sequences of integers
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['post_tokens'])
sequences = tokenizer.texts_to_sequences(df['post_tokens'])
X = pad_sequences(sequences, maxlen=maxlen)

# Define the target labels
y = pd.get_dummies(df['Label_cat']).values


In [ ]:
# Load the pre-trained GloVe embeddings
embedding_dict = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_dict[word] = vector

# Initialize the embedding matrix with pre-trained GloVe embeddings
word_index = tokenizer.word_index
num_words = min(max_words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, 100))
for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embedding_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [ ]:
# Define the model architecture
model = Sequential()
model.add(Embedding(num_words, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(GRU(64, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=RMSprop(lr=1e-4), loss='categorical_crossentropy', metrics=['acc'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
504/504 [==============================] - 28s 49ms/step - loss: 1.0673 - acc: 0.4202 - val_loss: 1.1600 - val_acc: 0.2268
Epoch 2/10
504/504 [==============================] - 25s 49ms/step - loss: 1.0490 - acc: 0.4278 - val_loss: 1.1610 - val_acc: 0.2268
Epoch 3/10
504/504 [==============================] - 23s 45ms/step - loss: 1.0396 - acc: 0.4310 - val_loss: 1.1643 - val_acc: 0.2953
Epoch 4/10
504/504 [==============================] - 25s 49ms/step - loss: 1.0373 - acc: 0.4309 - val_loss: 1.1619 - val_acc: 0.2970
Epoch 5/10
504/504 [==============================] - 24s 48ms/step - loss: 1.0322 - acc: 0.4327 - val_loss: 1.1635 - val_acc: 0.2953
Epoch 6/10
504/504 [==============================] - 22s 44ms/step - loss: 1.0324 - acc: 0.4352 - val_loss: 1.1642 - val_acc: 0.2275
Epoch 7/10
504/504 [==============================] - 24s 48ms/step - loss: 1.0323 - acc: 0.4291 - val_loss: 1.1556 - val_acc: 0.2988
Epoch 8/10
504/504 [==============================] - 29s 58ms